In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [3]:
from scipy.optimize import curve_fit

import tkinter as tk
from tkinter import filedialog

import os

from scipy import signal
import scipy.integrate as integrate
import matplotlib.ticker as ticker

# %matplotlib qt

In [4]:
%run functions
matplotlib.rcParams.update({'font.size': 25})
cm = pylab.get_cmap('tab10')
cm2 = pylab.get_cmap('Set1')
cm3 = pylab.get_cmap('turbo')

In [5]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.0e6*2*pi # 2.047e6
tauRF = 1/(Omega/2/pi)

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol

wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [62]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_path = filedialog.askopenfilename(multiple=True) # initialdir = dir_string
# print(file_path)

time       = []
T_aux_avg  = []
r2_v2_rlim = []

for i,j in enumerate(file_path):
    print(j)
    with load(j) as data:
        time.append(data['time'])
        T_aux_avg.append(data['temp'])
        r2_v2_rlim.append(data['r2_v2_rlim'])

/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC02_RF02.npz
/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC03_RF03.npz
/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC04_RF04.npz


In [ ]:
shape(time)

In [55]:
myRF = 0

for k in range(3):
    print(r2_v2_rlim[myRF][k+6][50]*1000) # [condition][dim][time]

R = r2_v2_rlim[myRF][6][50]*1e-3
L = r2_v2_rlim[myRF][8][50]*1e-3
aws = 0
print(R,L)
print((R+(1.48*aws/2))/(L+1.48*aws/2))

density = 1024 / (4/3*pi*R**2*L)

87.91131865763627
90.69376469714547
84.24134765828813
8.791131865763627e-05 8.424134765828814e-05
1.0435649606916877


In [37]:
def func2(t, t0, A, B,  C, D):
    return A*(t-t0)/(C+(t-t0)**2)**D + B
def func2abs(t, t0, A, B,  C, D):
    return A*(t-t0)/(C+abs(t-t0))**D + B
def func5(t, t0, A, B, C, k):
    return A*(t-t0) / (C+abs(t-t0)**k)**(1/k) + B
def dfunc5_dt(t,t0,A,B,C,k):
    return A*C*(abs(t-t0)**k+C)**( -(k+1)/k )
#     return A*(C+abs(t-t0)**k)**(-1/k)*(1 - ((t-t0)**2*abs(t-t0)**(k-2))/(C+abs(t-t0)**k) )

In [38]:
def func_lin(t,a,b):
    return a*t+b
def func_exp(x,a,b):
    return b*exp(-a*x)
def func_pow(t,A,B):
    return t**A * 10**B

In [61]:
for i,j in enumerate(file_path):
    print(i,j)

0 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0512_DC04_RF04.npz
1 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC02_RF02.npz
2 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC03_RF03.npz


In [56]:
to_fit = 0
my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
Urf = array([0, 0, 20.532292436697, 30.798438655045, 41.065, 55.35111, 57.40115, 59.45119, 61.50124, 63.55128, 65.60132, 67.65136, 69.70140, 71.75144,0, 0, 0, 0, 0,
             64.6, 62.5, 62.83, 63.03, 63.24, 63.44, 63.86, 64.06, 64.27, 64.47, 62.7262, 62.93, 63.14, 63.34, 63.55, 63.75, 63.96, 64.16, 64.37, 65.74])
q = array([0, 0, 0.2, 0.3, 0.4, 0.54, 0.56,  0.58,  0.6, 0.62,  0.64,  0.66,  0.68,  0.7, 0, 0, 0, 0, 0,
           0.63, 0.61, 0.612, 0.614, 0.616, 0.618, 0.622, 0.624, 0.66, 0.68, 0.611, 0.613, 0.615, 0.617, 0.619, 0.621, 0.623, 0.625, 0.627, 0.629])
Udc = array([0, 0, 1.3454211755646195, 3.062127421577093, 5.706, 11.10109, 12.0829, 13.12915, 14.24497, 15.43671, 16.71181, 18.07917, 19.54956, 21.13623, 0, 0, 0, 0, 0,
             16.0, 14.8, 14.95, 15.07, 15.19, 15.31, 15.56, 15.68, 15.81, 15.94, 14.89, 15.01, 15.13, 15.25, 15.37, 15.50, 15.62, 15.75, 15.87, 16.00])

beta_guess = 0.423
a = 0
for k in range(250):
    beta_guess = beta_continue_alamano(a,q,beta_guess)
    beta_guess = sqrt(beta_guess)

omega_z_2 = ( 2*pi*100e3 )**2 * array(Udc)
omega_x_2 = ( beta_guess*Omega/2 )**2
omega_r_2 = omega_x_2 - 0.5*omega_z_2
print(sqrt(omega_r_2[my_try])/2/pi)

x  = time[to_fit]-1e-3
fs = 1/diff(x)[1]

238882.79155328486


In [65]:
start_for_fit = 200
T_crit = 0.5
print('file to analyze')

to_fit = 2 #11 19 8
temp_to_end = 50

my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
print('>',to_fit,file_path[to_fit])
x  = time[to_fit]-1e-3
y  = T_aux_avg[to_fit]

max_f_r = sqrt(omega_r_2[my_try])/2/pi
b, a = signal.butter(8,  max_f_r/fs*2.01)
y_filt = signal.filtfilt(b, a, y, padlen=150)

index_100K = argmin(abs(y_filt-temp_to_end))
end_fit = index_100K

t0_first_guess = x[where(min(abs(y_filt-T_crit))==abs(y_filt-T_crit))][0] # 2.5e-3
B0 = log10(T_crit) # log10( y[where(min(abs(y-T_crit))==abs(y-T_crit))][0] )
A0 = log10(max(y_filt[start_for_fit:end_fit]) - B0)

p0 = [t0_first_guess,A0,B0,0.01,0.3]

# proper fit of the smoothed curve limiter up to threshold T
# popt, pcov = curve_fit(func5, x[start_for_fit:end_fit],
#                               log10(y_filt[start_for_fit:end_fit]),
#                        p0, maxfev= 25000,
#                        bounds=((-np.inf,-np.inf,-np.inf,0,0),(np.inf,np.inf,2,1.01,1)))
# print('func5',popt)

figname = 'testfitinlin0'
figure(figname,clear='True')
#xlim(0.8,6)
ax1 = subplot(111)
ax1.grid()
ax1.semilogy(x*1e3, y ,label=r'$T$',color='C0',lw=0.3)
ax1.semilogy(x*1e3, y_filt ,label=r'$T$ filtered',color='C1',ls=':',lw=3)
# ax2.vlines(x[end_fit]*1e3,1e-2,150,color='k')

# y_fit5 = func5(x[start_for_fit:end_fit],*popt)
# ax1.semilogy(x[start_for_fit:end_fit]*1e3,10**y_fit5,color='r',ls='--',label=r'$\psi$',lw=2)

nticks = 9
maj_loc = ticker.LogLocator(numticks=nticks)
min_loc = ticker.LogLocator(subs='all', numticks=nticks)
ax1.yaxis.set_major_locator(maj_loc)
ax1.yaxis.set_minor_locator(min_loc)

ax1.set_xlabel('t [ms]')
ax1.set_ylabel('T [K]')
# ax1.set_xlim(-0.1,4.3)
ax1.set_ylim(2e-3,300)
ax1.legend()
tight_layout()


# savefig(figname+'.eps',dpi=300)

file to analyze
> 2 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC04_RF04.npz


In [66]:
start_for_fit = 10
T_crit = 0.5
print('file to analyze')
not_fitted = 0
# 29 and 30
to_fit = 0
temp_to_end = [100, 100, 100, 50, 50, 50, 50, 50, 50,
               75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75,
               100, 100, 100, 100, 100, 100, 100, 100, 75, 150, 100, 100, 100, 100, 100]
popt_smooth = np.zeros((len(file_path),5))
for to_fit in range(len(file_path)):
    j = file_path[to_fit]
# retrive time and temperature from the curve
    print('>',to_fit,j)
    my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
    x  = time[to_fit]-1e-3
    y  = T_aux_avg[to_fit]

    # Smoothing signal with zero delay shift fit
    max_f_r = sqrt(omega_r_2[my_try])/2/pi
    b, a = signal.butter(8,  max_f_r/fs*1.01)
    y_filt = signal.filtfilt(b, a, y, padlen=150)
    
    # select index of temp where stop the fit
    # I stop the fit when T reaches a given T
    index_100K = argmin(abs(y_filt-temp_to_end[to_fit]))
    end_fit = index_100K

    # Fit parameters first guesses
    t0_first_guess = x[where(min(abs(y_filt-T_crit))==abs(y_filt-T_crit))][0] # 2.5e-3
    B0 = log10(T_crit) # log10( y[where(min(abs(y-T_crit))==abs(y-T_crit))][0] )
    A0 = log10(max(y_filt[start_for_fit:end_fit]) - B0)

    # set initial guess of fit parameters
    p0 = [t0_first_guess,A0,B0,0.01,0.3]

    # proper fit of the smoothed curve limiter up to threshold T
    popt, pcov = curve_fit(func5, x[start_for_fit:end_fit],
                                  log10(y_filt[start_for_fit:end_fit]),
                           p0, maxfev= 25000,
                           bounds=((-np.inf,-np.inf,-np.inf,0,0),(np.inf,np.inf,2,1.01,1)))
    print(popt)
    popt_smooth[to_fit,:] = popt
# popt_smooth = popt

file to analyze
> 0 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC02_RF02.npz
[2.32312746e-03 7.70757564e+00 3.28318087e-01 1.96284437e-01
 1.67227492e-01]
> 1 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC03_RF03.npz
[1.23048092e-03 2.92250919e+00 1.98790574e-01 5.44620201e-02
 2.92564770e-01]
> 2 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC04_RF04.npz
[0.02280041 1.90573408 0.27073818 0.00734952 0.50967884]


In [67]:
# when does fit crosses a given temperature
# compute delta T between this moment and t0
# computed above in the fit, from t=0
T_thre = 1e-2
fit_above_thresh = np.zeros((len(file_path)))
for to_fit in range(len(file_path)):
    j = file_path[to_fit]
    if to_fit!=0 and to_fit!=9 and to_fit!=10 and to_fit!=11:
        x = time[to_fit]-1e-3
        y_fit = func5(x,*popt_smooth[to_fit,:])
        fit_above_thresh[to_fit] = x[where(min(abs(10**y_fit-T_thre))==abs(10**y_fit-T_thre))][0]
print(fit_above_thresh)

delta_T_explosion = popt_smooth[:,0] - fit_above_thresh


[0.000e+00 6.245e-07 6.245e-07]


In [68]:
popt_rework = [popt_smooth[1:9],
              popt_smooth[12:20], # 7:15
              popt_smooth[20:]]

delta_T_explosion_rework = array([delta_T_explosion[1:9],
                            delta_T_explosion[12:20], # 7:15
                            delta_T_explosion[20:]])

#  3     4     5     6     7     8     9     10    11    12    13
# 31.0, 42.0, 55.4, 57.4, 59.5, 61.5, 63.6, 65.6, 67.7, 69.7, 71.8
#             0.54  0.56  0.58  0.6   0.62  0.64  0.66  0.68  0.7
Urf_rework = [[57.4, 59.5, 61.5, 63.6, 65.6, 67.7, 69.7, 71.8],
             [57.4, 59.5, 61.5, 63.6, 65.6, 67.7, 69.7, 71.8], # 57.4, 59.5, 
             [55.4, 57.4, 59.5, 61.5, 63.6, 65.6, 67.7, 69.7, 71.8,
                      64.6, 62.5, 62.83, 63.03, 63.24, 63.44, 63.86, 64.06, 64.27, 64.47,
                      62.7262, 62.93, 63.14, 63.34, 63.55, 63.75, 63.96, 64.16, 64.37, 65.74]]
q_rework = [[0.56,  0.58,  0.6,   0.62,  0.64,  0.66,  0.68,  0.7],
             [0.56,  0.58,  0.6,   0.62,  0.64,  0.66,  0.68,  0.7 ], # 57.4, 59.5, 
             [0.54, 0.56,  0.58,  0.6,   0.62,  0.64,  0.66,  0.68,  0.7,
                      0.63, 0.61, 0.612, 0.614, 0.616, 0.618, 0.622, 0.624, 0.626, 0.628,
                      0.611, 0.613, 0.615, 0.617, 0.619, 0.621, 0.623, 0.625, 0.627, 0.629]]
N_rework = [256, 512, 1024]


for k,l in enumerate(N_rework):
    print(f'fit_params_{l:04d}ions')
    np.savez( f'fit_params_{l:04d}ions', fit_param=popt_rework[k], Urf=Urf_rework[k], N=N_rework[k])

# func5(t, t0, A, B, C, k)
# with load('/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/fit_params_0256ions.npz') as data:
#     a = data['N']
# print(a)

fit_params_0256ions
fit_params_0512ions
fit_params_1024ions


/tmp/ipykernel_101673/798389194.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  delta_T_explosion_rework = array([delta_T_explosion[1:9],


In [18]:
figure('fit_parameters',clear=True)
ax1 = subplot(151)
for k in range(3):
    print(k)
    ax1.plot(Urf_rework[k],array(popt_rework[k])[:,0]*1e3-1,color=cm(k/10),ls=':',marker='P',label=N_rework[k])
    ax1.set_xlabel('$U_{RF}$ [V]')
    ax1.set_ylabel('$t_0$ [ms]')
    ax1.grid()
    ax1.legend(title='N',fontsize=12)
    ax2 = subplot(152)
    ax2.plot(Urf_rework[k],10**array(popt_rework[k])[:,1],color=cm(k/10),ls=':',marker='P',label=N_rework[k])
    ax2.set_xlabel('$U_{RF}$ [V]')
    ax2.set_ylabel('$10^A$ [K]')
    ax2.grid()
    ax2.legend(title='N',fontsize=12)
    ax3 = subplot(153)
    ax3.plot(Urf_rework[k],10**array(popt_rework[k])[:,2],color=cm(k/10),ls=':',marker='P',label=N_rework[k])
    ax3.set_xlabel('$U_{RF}$ [V]')
    ax3.set_ylabel('$10^B$ [K]')
    ax3.grid()
    ax3.legend(title='N',fontsize=12)
    ax4 = subplot(154)
    ax4.plot(Urf_rework[k],array(popt_rework[k])[:,3]*1e3,color=cm(k/10),ls=':',marker='P',label=N_rework[k])
    ax4.set_xlabel('$U_{RF}$ [V]')
    ax4.set_ylabel('$C$ [ms]')
    ax4.grid()
    ax4.legend(title='N',fontsize=12)
    ax5 = subplot(155)
    ax5.plot(Urf_rework[k],array(popt_rework[k])[:,4],color=cm(k/10),ls=':',marker='P',label=N_rework[k])
    ax5.set_xlabel('$U_{RF}$ [V]')
    ax5.set_ylabel('$k$ [1]')
    ax5.grid()
    ax5.legend(title='N',fontsize=12)
tight_layout()

0


ValueError: x and y must have same first dimension, but have shapes (8,) and (2,)

In [ ]:
C = []
D = []
x_forfit = [linspace(0.555,0.71,200),linspace(0.555,0.71,200),linspace(0.535,0.71,200)]
figname = 'fit_parameter_t0_log'
fig = figure(figname,clear=True)
fig.set_size_inches(11.7, 8.3)
ax1 = subplot(111)
for k in range(2):
    ax1.loglog(q_rework[k],delta_T_explosion_rework[k]*1e3,
                 color=cm(k/10),ls='',marker='P',ms=10,mew=1,label=N_rework[k])
    popt, pcov = curve_fit(func_lin, log10(array(q_rework[k])),log10(delta_T_explosion_rework[k]),maxfev= 5000)
    print(k,popt)
    C.append(10**popt[1])
    D.append(popt[0])
    ax1.loglog(x_forfit[k],func_pow(x_forfit[k],popt[0],popt[1])*1e3,
                 ls='--',color=cm(k/10))
    ax1.set_xlabel('$q_{x}$')
    ax1.set_ylabel('$\Delta t$ [ms]')
    ax1.grid()
    
k = 2
ax1.loglog(q_rework[2][:9],delta_T_explosion_rework[2][:9]*1e3,
                 color=cm(2/10),ls='',marker='P',ms=10,mew=1,label=N_rework[2])
popt, pcov = curve_fit(func_lin, log10(array(q_rework[2][:9])),
                       log10(delta_T_explosion_rework[2][:9]),
                       maxfev= 5000)
print(k,popt)
ax1.loglog(x_forfit[2],func_pow(x_forfit[2],popt[0],popt[1])*1e3,
                 ls='--',color=cm(2/10))
ax1.set_xticks([0.54,0.56,0.58,0.6,0.62,0.64,0.66,0.68,0.7])
# ax1.set_xticklabels(rotation=45, ha='right')
ax1.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax1.grid(visible=True, which='major', color='0.2', linestyle='-')
ax1.grid(visible=True, which='minor', color='0.7', linestyle='--')
ax1.legend(title='N',fontsize=18)
tight_layout()

savefig(figname+'.eps',dpi=300)
    
# fig = figure('fit_parameter_t0_bis',clear=True)
# fig.set_size_inches(11.7, 8.3)
# ax1 = subplot(111)
# for k in range(3):
#     ax1.plot(array(q_rework[k]),(array(popt_rework[k])[:,0]*1e3)**(-1/2),color=cm(k/4),
#              ls=':',marker='P',ms=10,label=N_rework[k])
#     popt, pcov = curve_fit(func_lin, array(q_rework[k]),(array(popt_rework[k])[:,0]*1e3)**(-1/2),maxfev= 15000)
#     ax1.plot(array(q_rework[k]),func_lin(array(q_rework[k]),popt[0],popt[1]),color=cm(k/3))
#     ax1.set_xlabel('$q_x$')
#     ax1.set_ylabel('$t_0^{-1/2}$ [ms$^{-1/2}$]')
#     ax1.grid()
#     ax1.legend(title='N',fontsize=12)
# tight_layout()

In [ ]:
# Around specific values

k = 2
x = hstack((q_rework[k][4],q_rework[k][9:]))
y = hstack((delta_T_explosion_rework[k][4]*1e3,delta_T_explosion_rework[k][9:]*1e3))

x, y = zip(*sorted(zip(x, y)))
figname='fit_parameter_t0_zoom'
fig = figure(figname,clear=True)
fig.set_size_inches(11.7, 8.3)
ax1 = subplot(111)
ax1.plot(x,y,
             color=cm(k/10),ls=':',marker='P',ms=10,mew=1,label=N_rework[k])
ax1.plot(linspace(0.61,0.63,200),func_pow(linspace(0.61,0.63,200),popt[0],popt[1])*1e3,
                 ls='--',color=cm(2/10))
ticks = np.linspace(0.61,0.63,11)
ax1.set_xticks(ticks)
ax1.xaxis.set_minor_formatter(mticker.ScalarFormatter())
plt.xticks(rotation = 45)
ax1.set_xlabel('$q_{x}$')
ax1.set_ylabel('$\Delta t$ [ms]')
ax1.grid()
ax1.legend(title='N',fontsize=18)
tight_layout()
savefig(figname+'.eps',dpi=300)

# Derivate study

# Windowing

In [69]:
c_light = 299792458
h_pl = 6.62607015*1e-34
h_pl_bar = h_pl/2/pi
lam_397 = 396.84620*1e-9
k=(2*pi)/lam_397
gamma_SP = 21.57*1e6
tau = 1/gamma_SP
Omega = 2.0e6*2*pi
tau_RF = 1/(Omega/(2*pi))

In [70]:
# Windowing temperature
window_samp = 200
x_win = [] # zeros((len(y)//window_samp,len(file_path)))
y_win = [] # zeros((len(y)//window_samp,len(file_path)))

for to_fit in range(len(file_path)):
#     print(to_fit)
    x_win.append([])
    y_win.append([])
    x = time[to_fit]-1e-3
    y = T_aux_avg[to_fit]
    for i in range(len(y)//window_samp-1):
        x_win[to_fit].append( (x[i*window_samp]+x[(i+1)*window_samp])/2 )
        y_win[to_fit].append( sum(y[i*window_samp:(i+1)*window_samp])/window_samp )
    
print(diff(x_win[0])[0]*1e3,tau_RF*window_samp*1e3)

0.10000000000000005 0.09999999999999999


In [72]:
# Plot windowed temperature
to_plot = 0
x  = time[to_plot]-1e-3
y  = T_aux_avg[to_plot]
figure('Windowed T',clear='True')
semilogy(x, y,':',color='y')
semilogy(x_win[to_plot], y_win[to_plot],'+')
# hlines(500,0,0.02)
grid()
tight_layout()

In [ ]:
figname = 'Windowed_dT_dt'
fig = figure(figname,clear='True')
fig.set_size_inches(11.7, 8.3)
ax = fig.add_subplot(111)
# ax.semilogy(x_win[:-1]*1e3, diff(y_win)/window_samp,'+')
for to_plot in [0]: # [21,23,25,27] [12,14,16,18] [1,3,5,7]
    my_try = int(file_path[to_plot][file_path[to_plot].find('DC')+2:file_path[to_plot].find('DC')+4])
    print(file_path[to_plot])
    ax.loglog(y_win[to_plot][:-1], diff(y_win[to_plot])/window_samp,
              marker='.',ms=20,mec='k',mew=0.3,ls='',
              label=f'{q[my_try]}')
# semilogy(x[::100]*1e3, y[::100],':',color='y')
# ax.set_xlabel('t [ms]')
ax.set_xlabel('T [K]')
ax.set_ylabel('$H=dT/dt$ [K/RF period]')
# ax.set_xlim(5e-3,1e-3)
ax.grid()
legend(title='$q_x$, $N=1024$',loc=0)
tight_layout()
# savefig(figname+'N1024_zoom'+'.eps',dpi=300)

In [ ]:
# Plot windowed temperature
# figure('Windowed T',clear='True')
# semilogy(x_win*1e3, y_win,'+')
# semilogy(x[::100]*1e3, y[::100],':',color='y')
# grid()
# tight_layout()
figname = 'Windowed_dT_dt'
fig = figure(figname,clear='True')
fig.set_size_inches(11.7, 8.3)
ax = fig.add_subplot(111)
# ax.semilogy(x_win[:-1]*1e3, diff(y_win)/window_samp,'+')
incr = 0
for to_plot in [0]: # [4,15,24]
    print(file_path[to_plot])
    my_try = int(file_path[to_plot][file_path[to_plot].find('DC')+2:file_path[to_plot].find('DC')+4])
    ax.loglog(y_win[to_plot][:-1], diff(y_win[to_plot])/window_samp,
              marker='.',ms=20,mec='k',mew=0.3,ls='',
              label=f'512')
    incr+=1
# semilogy(x[::100]*1e3, y[::100],':',color='y')
# ax.set_xlabel('t [ms]')
ax.set_xlabel('T [K]')
ax.set_ylabel('$H=\mathrm{d}T/\mathrm{d}t$ [K/RF period]')
# ax.set_xlim(3e-3,11)
ax.grid()
legend(title=f'$N$, $q_x={q[my_try]}$',loc=4)
tight_layout()

# savefig(figname+'q0.60'+'.eps',dpi=300)

In [89]:
# Plot windowed temperature
# figure('Windowed T',clear='True')
# semilogy(x_win*1e3, y_win,'+')
# semilogy(x[::100]*1e3, y[::100],':',color='y')
# grid()
# tight_layout

nv = 500
log_temps = linspace(-3.1,3,nv)
coolpow_05 = []
coolpow_1 = []
# coolpow_1_bis = []
coolpow_2 = []
coolpow_4 = []
coolpow_6 = []
# coolpow_8 = []
coolpow_10 = []
# coolpow_50 = []
for T in 10**log_temps:
    v_thermique = sqrt(2*kb*T/m_Ca)
    dv = 6*v_thermique/nv
    coolpow_05.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -Gam/2, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    coolpow_1.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -Gam, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_1_bis.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -Gam, k, 2*Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    coolpow_2.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -2*Gam, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    coolpow_4.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -4*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    coolpow_6.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -6*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_8.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -8*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    coolpow_10.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -10*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_50.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -50*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )

my_color=[7,2,6,3] # 
figname = 'Windowed_dT_dt_q_N'
fig = figure(figname,clear='True')
fig.set_size_inches(11.7, 8.3)
ax1 = fig.add_subplot(111)

# ax.semilogy(x_win[:-1]*1e3, diff(y_win)/window_samp,'+')
incr = 0
choosen_plot = [0,1,2]
for to_plot in choosen_plot: # [4,15,24] 1,12,21 3,14,23
    print(file_path[to_plot])
    my_try = int(file_path[to_plot][file_path[to_plot].find('DC')+2:file_path[to_plot].find('DC')+4])
    ax1.loglog(y_win[to_plot][:-1], diff(y_win[to_plot])/window_samp,
              marker='.',ms=14,mec='k',mew=0,ls='',color=cm3((incr+0.3)/4),
              label=f'{q[my_try]}')
    incr+=1
# loglog(0,0,' ',label=' ')
# to_plot = 20
# ax1.loglog(y_win[to_plot][:-1], diff(y_win[to_plot])/window_samp,
#           marker='.',ms=14,mec='k',mew=0,ls='',color=cm(4/8),
#           label=f'{N_rework[2]}')

# ax1.loglog(y_win[choosen_plot[0]][:-1],
#            diff(y_win[choosen_plot[0]][:])/window_samp,
#           marker='',ls=':',color=cm(0/8))
    
ax1.loglog(10**log_temps,coolpow_05,ls="-",color='k',label='$-0.5\Gamma$')
ax1.loglog(10**log_temps,coolpow_1,ls="--",color='k',label='$-\Gamma$')
# ax1.loglog(10**log_temps,coolpow_1_bis,ls="--",color='r',label='$-\Gamma$')
ax1.loglog(10**log_temps,coolpow_2,ls=":",color='xkcd:rouge',label='$-2\Gamma$')
ax1.loglog(10**log_temps,coolpow_4,ls=":",color='xkcd:gold',label='$-4\Gamma$')  
ax1.loglog(10**log_temps,coolpow_6,ls=":",color='xkcd:green',label='$-6\Gamma$')
# ax1.loglog(10**log_temps,coolpow_8,ls=":",color='xkcd:blue',label='$-8\Gamma$')  
ax1.loglog(10**log_temps,coolpow_10,ls=":",color='xkcd:purple',label='$-10\Gamma$')
# ax1.loglog(10**log_temps,coolpow_50,ls=":",color='xkcd:black',label='$-50\Gamma$') 
    
ax1.set_xlabel('T [K]')
ax1.set_ylabel('$H=\mathrm{d}T/\mathrm{d}t$ [K/RF period]')
ax1.grid()
ax1.grid(True, which="minor", ls=":", color='0.80')
ax1.legend(title=f'$N=1024$, $q_x$',loc=4,ncol=3)
ax1.set_xlim(2e-2,0.6e3)
ax1.set_ylim(1e-8,5e-1)

tight_layout()

savefig(figname+'qvar'+'.eps',dpi=300)

/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC02_RF02.npz
/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC03_RF03.npz
/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC04_RF04.npz


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [ ]:
for gamma in my_gamma
    for T in 10**log_temps:
        v_thermique = sqrt(2*kb*T/m_Ca)
        dv = 6*v_thermique/nv
        coolpow_gamma.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -Gam/2, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )

In [ ]:
window_samp = 200
window_size = tau_RF * window_samp
window_num = len(y)/window_samp
print(window_num)
print(len(y))

# From fluo-variations_optimal-temp.ipynb

# Functions used to describe ions

$\texttt{MB}$ is the Maxwell-Boltzmann distribution. It provides the probability to find an atom with a given velocity in a gas with temperature T.

$\texttt{pfl_dop} = \rho_{ee}$ is the atomic ray profile with Doppler effect. It provides the probability of excitation given Rabi frequency, detuning, lambda and velocity of atom. It is considered equal to the excited population, i.e the proportion of excited atoms.

$ \texttt{MB} = \sqrt{\frac{m}{2\pi k_BT}}\exp{-\frac{mv^2}{2k_BT}}$

$\texttt{pfl_dop} = \rho_{ee} = \frac{0.25\Omega_R^2}{0.5\Omega_R^2 + 0.25\Gamma^2 + \Delta^2} = \frac{A/2}{A+B + \Delta^2}$

In [22]:
# Doppler profile
def pfl_dop(v, delta, k, Rab, Gam):
    return .25*Rab**2/(0.5*Rab**2+.25*Gam**2+(delta-k*v)**2)

# Maxwell-Boltzmann distribution
def MB(v, T):
    kb = 1.38064852*1e-23  # Boltzman
    m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
    return (m_Ca/(np.pi*2*kb*T))**(1/2) * np.exp(-m_Ca*v**2/(2*kb*T)) # **1 car vitesse par rapport au laser compte seulement (1D)

# Probabilities product
# For a range of velocities,
# compute the prob_fluo
# for a given T, delta, Rab, Gam
def prob_fluo(vmin, vmax, nv, T, delta, k, Rab, Gam):
    nu = k*np.linspace(vmin, vmax, nv)
    return nu, pfl_dop(-nu/k, delta, k, Rab, Gam)*MB(nu/k, T)

def cool_power(vmin, vmax, nv, T, delta, k, Rab, Gam):
    nu = k*np.linspace(vmin, vmax, nv)
    return nu, pfl_dop(-nu/k, delta, k, Rab, Gam)*MB(nu/k, T)*nu

def T_lim(delta):
    return -0.5*hbar*Gam**2*(1+(2*delta/Gam)**2) / (4*delta)/kb

In [23]:
T = 0.01
v_thermique = sqrt(2*kb*T/m_Ca)
nv = 2000
dv = 6*v_thermique/nv
dist_MB = []
for v in linspace(-3*v_thermique,3*v_thermique,nv):
    dist_MB.append(MB(v, T))
print(sum(dist_MB)*dv)

0.9994781288008786


In [24]:
# Laser parameters
lam = 397e-9 # m
klam = 2*pi/lam # m^-1
Gam = 21570000.0 *2*pi
delta = Gam
Rab = Gam
I = 170 # W/m²

# Cooling energy

Work of cooling force is $\vec{F}\cdot\vec{v} = \hbar k\Gamma\rho_{22}(\vec{v})\cdot\vec{v}$. The energy is the work integrated over velocity $G$ :

$G = \hbar k\Gamma \int \rho_{22}(\vec{v})\cdot\vec{v} \mathrm{d}v$ .

In [ ]:
# Plot the MBxpfl-dop vs Temperature

N = N_rework[2]

Tmin = .0005
Tmax = 10
vmin = sqrt(2*kb*Tmin/m_Ca)
vmax = sqrt(2*kb*Tmax/m_Ca)
nv = 500
nu = -k*np.linspace(vmin,vmax,nv)
dv = mean(diff(nu))/k

Rab = Gam # or np.sqrt(3*lam**3*Gam/(4*np.pi**2*hbar*c) * I) 

T_moy = Tmax
v_moy = sqrt(8*kb*T_moy/pi/m_Ca)
vmoins = -10*v_moy
vplus = 10*v_moy

dv = (vplus-vmoins)/nv

pfl0 = []
pfl05 = []
pfl1 = []
pfl2 = []
coolpow = []

Temps = linspace(Tmin,200,nv)
for T in Temps:
    v_thermique = sqrt(2*kb*T/m_Ca)
    dv = 6*v_thermique/nv
#     pfl0.append( sum(prob_fluo(vmoins, vplus, nv, T, 0, k, Rab, Gam)[1])*dv*Gam * N )
    pfl05.append( sum(prob_fluo(vmoins, vplus, nv, T, -Gam/2, k, Rab, Gam)[1])*dv*h_pl_bar*k*Gam *2/3/kb*tau_RF ) # /kb/5e-7*2/3
    pfl1.append( sum(prob_fluo(vmoins, vplus, nv, T, -Gam, k, Rab, Gam)[1])*dv*h_pl_bar*k*Gam *2/3/kb*tau_RF )
    coolpow.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -Gam/2, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     pfl2.append( sum(prob_fluo(vmoins, vplus, nv, T, Gam, k, Rab, Gam)[1])*dv*Gam * N )
# pfl3 = pfl_dop(nu/k, +3*Gam, k, Rab, Gam)

fig = plt.figure('MBxDop prof fluo tot',clear='True')
fig.set_size_inches(11.7, 8.3)
ax = fig.add_subplot(111)

# ax.plot(Temps, array(pfl0)*1e-9, '-',color='xkcd:green',label=r'$\delta = 0$')
ax.plot(Temps, array(pfl05)*1e-9, ':',color='xkcd:black',label=r'$\delta = -\Gamma/2$')
# ax.plot(Temps, array(pfl2)*1e-9, '-',color='xkcd:blue',label=r'$\delta = \Gamma$')
ax.plot(Temps, array(pfl1)*1e-9, '--',color='xkcd:red',label=r'$\delta = -\Gamma$')
# ax.plot((nu/k)**2*m_Ca/2/kb, pfl3, '-',color='xkcd:black',label=r'$\delta = \Gamma$')

# ax.hlines((pfl2[0]-sqrt(pfl2[0]))*1e-9,0,1,color='xkcd:red',ls=':',lw = 3)
# ax.hlines((pfl2[0]+sqrt(pfl2[0]))*1e-9,0,1,color='xkcd:red',ls=':',lw = 3)
# ax.hlines(pfl0[0]-sqrt(pfl0[0]),0,0.1,color='xkcd:green',ls=':',lw = 3)
# ax.hlines(pfl05[0]-sqrt(pfl05[0]),0,0.1,color='xkcd:blue',ls=':',lw = 3)


ax.set_xlabel(r'T [K]')
ax.set_ylabel(r'Cooling energy $G$ [K/RF period]')
ax.set_xlim(-0.02,2.22)
# ax.set_ylim(12,46)

ax.grid()
ax.legend(title='detuning',loc=1,fontsize=15)

plt.tight_layout()
# subplots_adjust(top=0.96,
# bottom=0.135,
# left=0.115,
# right=0.965,
# hspace=0.2,
# wspace=0.2)

In [ ]:
fig = plt.figure('d/dt MBxDop prof fluo tot',clear='True')
fig.set_size_inches(11.7, 8.3)
ax = fig.add_subplot(111)

ax.plot(Temps[:-1], diff(array(pfl0))*1e-9, '-',color='xkcd:green',label=r'$\delta = 0$')
ax.plot(Temps[:-1], diff(array(pfl05))*1e-9, ':',color='xkcd:black',label=r'$\delta = -\Gamma/2$')
ax.plot(Temps[:-1], diff(array(pfl1))*1e-9, '--',color='xkcd:red',label=r'$\delta = -\Gamma$')
ax.grid()
ax.set_xlim(0,1)
ax.legend(title='detuning',loc=1,fontsize=15)

# alpha variation

In [ ]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_path_alpha = filedialog.askopenfilename(multiple=True) # initialdir = dir_string
# print(file_path)

time_alpha       = []
T_aux_avg_alpha  = []
r2_v2_rlim_alpha      = []

for i,j in enumerate(file_path_alpha):
    print(j)
    with load(j) as data:
        time_alpha.append(data['time'])
        T_aux_avg_alpha.append(data['temp'])
        r2_v2_rlim_alpha.append(data['r2_v2_rlim'])

In [ ]:
to_fit = 2
my_try = int(file_path_alpha[to_fit][file_path_alpha[to_fit].find('DC')+2:file_path_alpha[to_fit].find('DC')+4])
Urf_alpha = array([65.70, 0, 65.70, 65.70,0, 65.70,  65.70, 0, 65.70, 65.70])
q_alpha = array([0.64, 0, 0.64, 0.64, 0, 0.64, 0.64, 0, 0.64, 0.64])
Udc_alpha = array([10.027, 0, 12, 13.848, 0, 15.85, 17.669, 0, 19.7, 21.49])
# alpha = [0.49999436, 0.76323903, 1.08844661, 1.50041269] approx !!

beta_guess_alpha = 0.423
a = 0
for k in range(250):
    beta_guess_alpha = beta_continue_alamano(a,q_alpha,beta_guess_alpha)
    beta_guess_alpha = sqrt(beta_guess_alpha)

omega_z_2_alpha = ( 2*pi*100e3 )**2 * array(Udc_alpha)
omega_x_2_alpha = ( beta_guess_alpha*Omega/2 )**2
omega_r_2_alpha = omega_x_2_alpha - 0.5*omega_z_2_alpha
print(sqrt(omega_r_2_alpha[my_try])/2/pi)

x  = time_alpha[to_fit]-1e-3
fs = 1/diff(x)[1]

In [ ]:
alpha = []
R1 = []
R2 = []
L = []
for myRF in range(len(file_path_alpha)):
    R1.append(r2_v2_rlim_alpha[myRF][6][50]*1000)
    R2.append(r2_v2_rlim_alpha[myRF][6][50]*1000)
    L.append(r2_v2_rlim_alpha[myRF][8][50]*1000)
    alpha.append((R1[myRF]+R2[myRF])/(2*L[myRF]))
V = 4/3*pi*((array(R1)+array(R2))/2)**2*array(L)# aws = 0
# print((R+(1.48*aws/2))/(L+1.48*aws/2))
print(alpha)

In [ ]:
start_for_fit = 200
T_crit = 0.5
print('file to analyze')

to_fit = 3 #11 19 8
temp_to_end = 75

my_try = int(file_path_alpha[to_fit][file_path_alpha[to_fit].find('DC')+2:file_path_alpha[to_fit].find('DC')+4])
print('>',to_fit,file_path_alpha[to_fit])
x  = time_alpha[to_fit]-1e-3
y  = T_aux_avg_alpha[to_fit]

max_f_r = sqrt(omega_r_2_alpha[my_try])/2/pi
b, a = signal.butter(8,  max_f_r/fs*1.01)
y_filt = signal.filtfilt(b, a, y, padlen=150)

index_100K = argmin(abs(y_filt-temp_to_end))
end_fit = index_100K

t0_first_guess = x[where(min(abs(y_filt-T_crit))==abs(y_filt-T_crit))][0] # 2.5e-3
B0 = log10(T_crit) # log10( y[where(min(abs(y-T_crit))==abs(y-T_crit))][0] )
A0 = log10(max(y_filt[start_for_fit:end_fit]) - B0)



p0 = [t0_first_guess,A0,B0,0.01,0.3]

# proper fit of the smoothed curve limiter up to threshold T
popt, pcov = curve_fit(func5, x[start_for_fit:end_fit],
                              log10(y_filt[start_for_fit:end_fit]),
                       p0, maxfev= 25000,
                       bounds=((-np.inf,-np.inf,-np.inf,0,0),(np.inf,np.inf,2,1.01,1)))
print('func5',popt)

figname = 'testfitinlin0'
figure(figname,clear='True')
#xlim(0.8,6)
ax2 = subplot(111)
ax2.grid()
ax2.semilogy(x*1e3, y ,label=r'$T$',color='C0',lw=0.3)
ax2.semilogy(x*1e3, y_filt ,label=r'$T$ filtered',color='C1',ls=':',lw=3)
ax2.vlines(x[end_fit]*1e3,1e-2,150,color='k')

y_fit5 = func5(x[start_for_fit:end_fit],*popt)
ax2.semilogy(x[start_for_fit:end_fit]*1e3,10**y_fit5,color='r',ls='--',label=r'$\psi$',lw=2)

nticks = 9
maj_loc = ticker.LogLocator(numticks=nticks)
min_loc = ticker.LogLocator(subs='all', numticks=nticks)
ax2.yaxis.set_major_locator(maj_loc)
ax2.yaxis.set_minor_locator(min_loc)

ax2.set_xlabel('t [ms]')
ax2.set_ylabel('T [K]')
# ax2.set_xlim(-0.3,5.6)
# ax2.set_ylim(2e-3,300)
tight_layout()
# savefig(figname+'.eps',dpi=300)

In [ ]:
# Fit all curves
# Smooth data temperature
# guess first parameter set
# fit smoothed data up to threshold temperature
# much more consistent than below

popt_smooth_alpha = np.zeros((len(file_path_alpha),5))
temp_to_end = [75, 75, 75, 75, 75, 75, 75, 75] # Temperature to stop the fit
# 100, 100, 100, 100, 100, 100, 100, 100, 100   75, 75, 75, 75, 75, 75, 75, 75, 75
start_for_fit = 200
T_crit = 0.5
print('file to analyze')
not_fitted = 0
for to_fit in range(len(file_path_alpha)):
    j = file_path_alpha[to_fit]
# retrive time and temperature from the curve
    print('>',to_fit,j)
    my_try = int(file_path_alpha[to_fit][file_path_alpha[to_fit].find('DC')+2:file_path_alpha[to_fit].find('DC')+4])
    x  = time_alpha[to_fit]-1e-3
    y  = T_aux_avg_alpha[to_fit]

    # Smoothing signal with zero delay shift fit
    max_f_r = sqrt(omega_r_2_alpha[my_try])/2/pi
    b, a = signal.butter(8,  max_f_r/fs*2.01)
    y_filt = signal.filtfilt(b, a, y, padlen=150)

    # select index of temp where stop the fit
    # I stop the fit when T reaches a given T
    index_100K = argmin(abs(y_filt-temp_to_end[to_fit]))
    end_fit = index_100K

    # Fit parameters first guesses
    t0_first_guess = x[where(min(abs(y_filt-T_crit))==abs(y_filt-T_crit))][0] # 2.5e-3
    B0 = log10(T_crit) # log10( y[where(min(abs(y-T_crit))==abs(y-T_crit))][0] )
    A0 = log10(max(y_filt[start_for_fit:end_fit]) - B0)

    # set initial guess of fit parameters
    p0 = [t0_first_guess,A0,B0,0.01,0.3]

    # proper fit of the smoothed curve limiter up to threshold T
    popt, pcov = curve_fit(func5, x[start_for_fit:end_fit],
                                  log10(y_filt[start_for_fit:end_fit]),
                           p0, maxfev= 25000,
                           bounds=((-np.inf,-np.inf,-np.inf,0,0),(np.inf,np.inf,2,1.01,1)))
    print(popt)
    popt_smooth_alpha[to_fit,:] = popt

print('total amount of non fitted curves',not_fitted)

In [ ]:
# when does fit crosses a given temperature
# compute delta T between this moment and t0
# computed above in the fit, from t=0
T_thre = 1e-2
fit_above_thresh_alpha = np.zeros((len(file_path_alpha)))
for to_fit in range(len(file_path_alpha)):
    j = file_path_alpha[to_fit]
    x = time_alpha[to_fit]-1e-3
    y_fit = func5(x,*popt_smooth_alpha[to_fit,:])
    fit_above_thresh_alpha[to_fit] = x[where(min(abs(10**y_fit-T_thre))==abs(10**y_fit-T_thre))][0]
print(fit_above_thresh_alpha)

delta_T_explosion_alpha = popt_smooth_alpha[:,0] - fit_above_thresh_alpha
print(delta_T_explosion_alpha)

In [ ]:
q_rework = [0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64, 0.64]
figure('fit_parameters',clear=True)
ax1 = subplot(151)
ax1.plot(alpha,array(popt_smooth_alpha)[:,0]*1e3-1,color=cm(6/10),ls=':',marker='P',label=N_rework[2])
ax1.set_xlabel(r'$\alpha $')
ax1.set_ylabel('$t_0$ [ms]')
ax1.grid()
ax1.legend(title='N',fontsize=12)
ax2 = subplot(152)
ax2.plot(alpha,10**array(popt_smooth_alpha)[:,1],color=cm(6/10),ls=':',marker='P',label=N_rework[2])
ax2.set_xlabel(r'$\alpha $')
ax2.set_ylabel('$10^A$ [K]')
ax2.grid()
ax2.legend(title='N',fontsize=12)
ax3 = subplot(153)
ax3.plot(alpha,10**array(popt_smooth_alpha)[:,2],color=cm(6/10),ls=':',marker='P',label=N_rework[2])
ax3.set_xlabel(r'$\alpha $')
ax3.set_ylabel('$10^B$ [K]')
ax3.grid()
ax3.legend(title='N',fontsize=12)
ax4 = subplot(154)
ax4.plot(alpha,array(popt_smooth_alpha)[:,3]*1e3,color=cm(6/10),ls=':',marker='P',label=N_rework[2])
ax4.set_xlabel(r'$\alpha $')
ax4.set_ylabel('$C$ [ms]')
ax4.grid()
ax4.legend(title='N',fontsize=12)
ax5 = subplot(155)
ax5.plot(alpha,array(popt_smooth_alpha)[:,4],color=cm(6/10),ls=':',marker='P',label=N_rework[2])
ax5.set_xlabel(r'$\alpha $')
ax5.set_ylabel('$k$ [1]')
ax5.grid()
ax5.legend(title='N',fontsize=12)
tight_layout()

In [ ]:
C = []
D = []
x_forfit = [linspace(0.555,0.71,200),linspace(0.555,0.71,200),linspace(0.535,0.71,200)]
figname = 'fit_parameter_t0_log_alphavar'
fig = figure(figname,clear=True)
fig.set_size_inches(11.7, 8.3)
ax1 = subplot(111)
ax1.plot(alpha,delta_T_explosion_alpha*1e3,
             color=cm(6/10),ls=':',marker='P',ms=10,mew=1,label=N_rework[2])
# popt, pcov = curve_fit(func_lin, log10(array(popt_smooth)),log10(array(popt_smooth)[:,0]),maxfev= 5000)
print(k,popt)
ax1.set_xlabel(r'$\alpha$')
ax1.set_ylabel('$\Delta t$ [ms]')
ax1.grid()
ax1.legend(title='N',fontsize=18)
tight_layout()

savefig(figname+'.eps',dpi=300)

In [ ]:
C = []
D = []
x_forfit = [linspace(0.555,0.71,200),linspace(0.555,0.71,200),linspace(0.535,0.71,200)]
figname = 'fit_parameter_t0_log_volume'
fig = figure(figname,clear=True)
fig.set_size_inches(11.7, 8.3)
ax1 = subplot(111)
ax1.plot(V,delta_T_explosion_alpha*1e3,
             color=cm(6/10),ls=':',marker='P',ms=10,mew=1,label=N_rework[2])
# popt, pcov = curve_fit(func_lin, log10(array(popt_smooth)),log10(array(popt_smooth)[:,0]),maxfev= 5000)
print(k,popt)
ax1.set_xlabel(r'$V$ [µm$^3$]')
ax1.set_ylabel('$\Delta t$ [ms]')
ax1.grid()
ax1.legend(title='N',fontsize=18)
tight_layout()

savefig(figname+'.eps',dpi=300)

In [ ]:
R1(R1+R2)/(2*L)

In [ ]:
# Windowing temperature
window_samp_alpha = 200
x_win_alpha = [] # zeros((len(y)//window_samp,len(file_path)))
y_win_alpha = [] # zeros((len(y)//window_samp,len(file_path)))

for to_fit in range(len(file_path_alpha)):
#     print(to_fit)
    x_win_alpha.append([])
    y_win_alpha.append([])
    x = time_alpha[to_fit]-1e-3
    y = T_aux_avg_alpha[to_fit]
    for i in range(len(y)//window_samp_alpha-1):
        x_win_alpha[to_fit].append( (x[i*window_samp_alpha]+x[(i+1)*window_samp_alpha])/2 )
        y_win_alpha[to_fit].append( sum(y[i*window_samp_alpha:(i+1)*window_samp_alpha])/window_samp_alpha )
    
print(diff(x_win_alpha[0])[0]*1e3,tau_RF*window_samp_alpha*1e3)

In [ ]:
# Plot windowed temperature
to_plot = 4
x  = time_alpha[to_plot]-1e-3
y  = T_aux_avg_alpha[to_plot]
figure('Windowed T',clear='True')
semilogy(x, y,':',color='y')
semilogy(x_win_alpha[to_plot], y_win_alpha[to_plot],'+')
hlines(500,0,0.02)
grid()
tight_layout()

In [ ]:
# Plot windowed temperature
# figure('Windowed T',clear='True')
# semilogy(x_win*1e3, y_win,'+')
# semilogy(x[::100]*1e3, y[::100],':',color='y')
# grid()
# tight_layout

nv = 500
log_temps = linspace(-3.1,3,nv)
coolpow_05_alpha = []
coolpow_1_alpha = []
# coolpow_1_bis = []
coolpow_2_alpha = []
# coolpow_4 = []
# coolpow_6 = []
# coolpow_8 = []
# coolpow_10 = []
# coolpow_50 = []
for T in 10**log_temps:
    v_thermique = sqrt(2*kb*T/m_Ca)
    dv = 6*v_thermique/nv
    coolpow_05_alpha.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -Gam/2, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    coolpow_1_alpha.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -Gam, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_1_bis.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -Gam, k, 2*Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
    coolpow_2_alpha.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -2*Gam, klam, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_4.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -4*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_6.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -6*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_8.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -8*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_10.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -10*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )
#     coolpow_50.append( sum(cool_power(-3*v_thermique, 3*v_thermique, nv, T, -50*Gam, k, Rab, Gam)[1])*dv*h_pl_bar*Gam/kb*tau_RF )

my_color=[7,2,6,3] # 
figname = 'Windowed_dT_dt_alpha'
fig = figure(figname,clear='True')
fig.set_size_inches(11.7, 8.3)
ax1 = fig.add_subplot(111)

# ax.semilogy(x_win[:-1]*1e3, diff(y_win)/window_samp,'+')
incr = 0
choosen_plot = [0,1,2,3,4,5,6]
for to_plot in choosen_plot: # [4,15,24] 1,12,21 3,14,23
    print(file_path_alpha[to_plot])
    my_try = int(file_path_alpha[to_plot][file_path_alpha[to_plot].find('DC')+2:file_path_alpha[to_plot].find('DC')+4])
    ax1.loglog(y_win_alpha[to_plot][:-1], diff(y_win_alpha[to_plot])/window_samp_alpha,
              marker='.',ms=14,mec='k',mew=0,ls='',color=cm3(to_plot/7),
              label=f'{alpha[to_plot]:.2f}')
    incr+=1
# to_plot = 20
# ax1.loglog(y_win[to_plot][:-1], diff(y_win[to_plot])/window_samp,
#           marker='.',ms=14,mec='k',mew=0,ls='',color=cm(4/8),
#           label=f'{N_rework[2]}')

ax1.loglog(y_win_alpha[choosen_plot[0]][205:-251],
           diff(y_win_alpha[choosen_plot[0]][205:-250])/window_samp_alpha,
          marker='',ls=':',color=cm(0/8))
ax1.loglog(y_win_alpha[choosen_plot[1]][115:-200],
           diff(y_win_alpha[choosen_plot[1]][115:-199])/window_samp_alpha,
          marker='',ls=':',color=cm(1/8))
ax1.loglog(y_win_alpha[choosen_plot[2]][35:-80],
           diff(y_win_alpha[choosen_plot[2]][35:-79])/window_samp_alpha,
          marker='',ls=':',color=cm(2/8))
    
ax1.loglog(10**log_temps,coolpow_05_alpha,ls="-",color='k',label='$-0.5\Gamma$')
ax1.loglog(10**log_temps,coolpow_1_alpha,ls="--",color='k',label='$-\Gamma$')
# ax1.loglog(10**log_temps,coolpow_1_bis,ls="--",color='r',label='$-\Gamma$')
ax1.loglog(10**log_temps,coolpow_2_alpha,ls=":",color='k',label='$-2\Gamma$')
# ax1.loglog(10**log_temps,coolpow_4,ls=":",color='xkcd:yellow',label='$-4\Gamma$')  
# ax1.loglog(10**log_temps,coolpow_6,ls=":",color='xkcd:green',label='$-6\Gamma$')
# ax1.loglog(10**log_temps,coolpow_8,ls=":",color='xkcd:blue',label='$-8\Gamma$')  
# ax1.loglog(10**log_temps,coolpow_10,ls=":",color='xkcd:purple',label='$-10\Gamma$')
# ax1.loglog(10**log_temps,coolpow_50,ls=":",color='xkcd:black',label='$-50\Gamma$') 
    
ax1.set_xlabel('T [K]')
ax1.set_ylabel('$H=\mathrm{d}T/\mathrm{d}t$ [K/RF period]')
ax1.grid()
ax1.grid(True, which="minor", ls=":", color='0.80')
ax1.legend(title=r'$\alpha $',loc=4,ncol=2)

ax1.grid(True, which="minor", ls=":", color='0.80')
ax1.set_ylim(1e-7,0.55)
ax1.set_xlim(6e-3,520)

tight_layout()
# 
savefig(figname+'.eps',dpi=300)